In [ ]:
!pip install --upgrade google-cloud-vision
!pip install --upgrade google-cloud
!pip install --upgrade google-auth

In [ ]:
 
import requests 
from bs4 import BeautifulSoup 
import os
import io
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
 
from oauth2client.client import GoogleCredentials
from __future__ import print_function
 

from google.cloud import vision
 

In [ ]:
# Imports Credential File:
credential_path = "assignment2-344813-d48ba7b52eb3.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path
print("Service Account Key: {}".format(os.environ["GOOGLE_APPLICATION_CREDENTIALS"]))

Service Account Key: assignment2-344813-d48ba7b52eb3.json


In [ ]:
# Instantiates a client
client = vision.ImageAnnotatorClient()

In [ ]:
 
from PIL import Image, ImageDraw
def detect_face(face_file, max_results=4):
    """Uses the Vision API to detect faces in the given file.

    Args:
        face_file: A file-like object containing an image with faces.

    Returns:
        An array of Face objects with information about the picture.
    """
    client = vision.ImageAnnotatorClient()

    content = face_file.read()
    image = vision.Image(content=content)

    return client.face_detection(
        image=image, max_results=max_results).face_annotations

In [ ]:
def highlight_faces(image, faces, output_filename):
    """Draws a polygon around the faces, then saves to output_filename.

    Args:
      image: a file containing the image with the faces.
      faces: a list of faces found in the file. This should be in the format
          returned by the Vision API.
      output_filename: the name of the image file to be created, where the
          faces have polygons drawn around them.
    """
    im = Image.open(image)
    draw = ImageDraw.Draw(im)
    # Sepecify the font-family and the font-size
    for face in faces:
        box = [(vertex.x, vertex.y)
               for vertex in face.bounding_poly.vertices]
        draw.line(box + [box[0]], width=5, fill='#00ff00')
        # Place the confidence value/score of the detected faces above the
        # detection box in the output image
        draw.text(((face.bounding_poly.vertices)[0].x,
                   (face.bounding_poly.vertices)[0].y - 30),
                  str(format(face.detection_confidence, '.3f')) + '%',
                  fill='#FF0000')
    im.save(output_filename)

In [ ]:
def detect_logos(path):
    """Detects logos in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.logo_detection(image=image)
    logos = response.logo_annotations
    print('Logos:')

    for logo in logos:
        print(logo.description)

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

In [ ]:
def main(input_filename, output_filename, max_results):
    with open(input_filename, 'rb') as image:
        faces = detect_face(image, max_results)
        print('Found {} face{}'.format(
            len(faces), '' if len(faces) == 1 else 's'))

        print('Writing to file {}'.format(output_filename))
        detect_logos(input_filename)
        # Reset the file pointer, so we can read the file again
        image.seek(0)
        highlight_faces(image, faces, output_filename)

In [ ]:
 

path_to_img1 = '/content/img_01.png'
path_to_img2 = '/content/img_02.jpg'
path_to_img3 = '/content/img_03.jpg'
 

In [ ]:
out1='out1.jpg'
out2='out2.jpg'
out3='out3.jpg'


In [ ]:
main(path_to_img1, out1, 10)

Found 0 faces
Writing to file out1.jpg
Logos:
Amazon


In [ ]:
main(path_to_img2, out2, 10)

Found 6 faces
Writing to file out2.jpg
Logos:


In [ ]:
main(path_to_img3, out3, 10)

Found 2 faces
Writing to file out3.jpg
Logos:
Kid Cuisine
